<!-- # Title here {-} -->

Per Halvorsen   \
[pmhalvor@uio.no](mailto:pmhalvor@uio.no) \
GEO4460         

# Introduction

In this paper, we build a cost-distance path between two points based on elevation and land cover data, such as snow and lakes.

The data used to build this analysis comes from the [N50 Kartdata](https://kartkatalog.geonorge.no/metadata/n50-kartdata/ea192681-d039-42ec-b1bc-f3ce04c189ac) and the [Satellittdata - Sentinel-2 - Skyfritt opptak Norge 2022](https://kartkatalog.geonorge.no/metadata/satellittdata-sentinel-2-skyfritt-opptak-norge-2022/2e996bf2-9b7b-4700-8a26-c1a8a274c136?search=satellittdata). The steps taken to generate the rasters and feature layers were executed in ArcGIS Pro.

**Cost distance** is an estimate of how difficult it is to travel from one point to another, taking into account the terrain and land cover. 
The cost-distance path is the route that minimizes the total cost of travel between two points.
The desired output of this project is the optimal path between two DNT cabins in Vestlandet, a municipality on the west coast of Norway.

Topics we explore in this analysis include:

- Joining satellite-generated orthophotos as an image mosaic
- Filtering satellite data based on values of particular bands
- Extracting barriers from land cover data
- Generating slope from elevation data
- Calculating an accumulated cost raster
- Joining rasters through basic arithmetic 
- Finding the least-cost path between two points


# Data

Inspired by the data presented in the lab and a recent ski trip from the previous winter, we decided to focus this project on the Finse region of Vestlandet.  
Instead of reusing the data from the lab, we found similar datasets on [Kartkatalog](https://kartkatalog.geonorge.no/) containing all the information needed to build this analysis.

The data sources used in this project were:
- [N50 Kartdata](https://kartkatalog.geonorge.no/metadata/n50-kartdata/ea192681-d039-42ec-b1bc-f3ce04c189ac)
- [Satellittdata - Sentinel-2 - Skyfritt opptak Norge 2022](https://kartkatalog.geonorge.no/metadata/satellittdata-sentinel-2-skyfritt-opptak-norge-2022/2e996bf2-9b7b-4700-8a26-c1a8a274c136?search=satellittdata)
- [Norges Kart](https://www.norgeskart.no/)

## [N50 Kartdata](https://kartkatalog.geonorge.no/metadata/n50-kartdata/ea192681-d039-42ec-b1bc-f3ce04c189ac)

The N50 Kartdata contains a variety of rasters, polylines, and vectors, including elevation and land cover, among other geographic features.  
Given both cabins we'll be observing lie in the same municipality, we only needed to download the data for the Vestlandet region.

In the [Method](#method) section, we will go through how an elevation raster was used to create a slope raster, serving as the foundation for our cost layer.  
We'll also touch on how to extract lakes and rivers as barrier features to be used in the distance accumulation calculation.

## [Sentinel-2](https://kartkatalog.geonorge.no/metadata/satellittdata-sentinel-2-skyfritt-opptak-norge-2022/2e996bf2-9b7b-4700-8a26-c1a8a274c136?search=satellittdata)

Similar to the instructions from the lab, we'll use the cloud-free Sentinel-2 satellite imagery to create an orthophoto mosaic of the area we're interested in.  
This is necessary since the data is only available for download as cells of roughly 60km x 60km, and the area of interest spans across multiple cells.

These data are formatted as a 3-band `.tif` raster, with a red band, a green band, and a blue band.
Given that white is represented by max values for all three bands at once, we only really need one of these bands to build our snow mask.
More on that process in the [Method](#method) section.


## [Norges Kart](https://www.norgeskart.no/)

This source was only used to find the precise coordinates of the [Finse](https://www.norgeskart.no/#!?project=norgeskart&layers=1002&zoom=13&lat=6741397.04&lon=90229.32&markerLat=6741397.038351989&markerLon=90229.32121188036&p=searchOptionsPanel&sok=Finse) and [Krækkja](https://www.norgeskart.no/#!?project=norgeskart&layers=1002&zoom=13&lat=6723867.99&lon=99656.47&sok=kr%C3%A6kkja&markerLat=6723867.985645048&markerLon=99656.46813979524&p=searchOptionsPanel) cabins.  
The latitude and longitude coordinates for both cabins were used to create Geo-Points. The process for this is described in the [Method](#method) section.


# Method

- Building satellite image mosaic
- Filtering single band for snow mask
- Extracting barriers from land cover data
- Creating Geo-Points for the cabins
- Generating slope from elevation data
- Calculating an accumulated cost raster
- Joining rasters through basic arithmetic
- Finding the least cost path between two points


## Satellite data
### Building the image mosaic
Once we had downloaded the data from our source, we loaded only `Band_1` of both our current map in ArcGIS Pro. 
We used the **Mosaic To New Raster** tool to combine multiple satellite images into a single raster dataset. 
To do so, we needed to specify the input rasters, output location, and desired pixel type. 
This step simplifies downstream processing to only handling one raster instead of two. 

### Filtering snow
As previously mentioned, we know that a pixel that is white must have a max value in all three bands. 
A good approximation of where snow is then the pixels that have max values in one of the bands.
It should be noteed this is a simplification, to avoid having to run calcuations on all three bands, which would be even more precise. 
For our analysis, we deemed this simplification acceptable.

The min and max values of digital image data is typically 0 and 255, respectively.
So we decided to use a threshold of 200, meaning that any pixel with a value greater than 200 in the red band should most likely be classified as snow.

To filter the snow pixels, we used the **Raster Calculator** tool.
The simple arthmetric expression we used was:

$$
\text{snow} = \text{Band}_1 > 200
$$


One drawback of this simpler approach is how shadows might be handled. 
Typically, there are not many shadows present in cloud-free images, but that is not always the case, as we noticed a few in our data, especially in steep areas.
Compensating for this is however outside the scope of this project, as we'll rely on high local slopes around these shadow regions to make them less likely to be included in the final least-cost path.
In a later experiment, it would be interesting to further explore the percentage of misclassified shadow pixels in the snow mask.


## N50 data 
### Extracting barriers from land cover data

In our N50 Kartdata, there exists a layer called `Arealdekke`, which contains information about the land cover types in the area.
We were able to select and extrac the lakes and rivers from this layer using the **Select By Attributes** and **Extract By Attributes** tools.
The categories we selected were `innsjø` (lake) and `elv` (river). <!--TODO double check -->

### Generating slope from elevation data
The N50 data also contains a layer called XXXX, which contains contour lines representing the elevation of the terrain.
From this, we were able to generate a DEM, or a digital elevation model, using the **Feature To Raster** tool.
A DEM raster is a continuous surface representing the elevation of the terrain.

Once we had the DEM, we were able to generate a raster for the slope using the **Surface Parameters** tool.
The slope raster represents the steepness of the terrain, which is important for calculating the cost of travel across the landscape.
The higher the slope, the more difficult it is to travel across the terrain, and thus the higher the cost. 

A more in-depth analysis would also consider directionality.
However, for this project, we stuck to the simpolistic interpretation of straight-forward slope as a cost factor.

## Coordinates to Geo-Points
The cabins we'll be observing are located at the following coordinates:
- lat=6742013.21&lon=89450.51


## Calculating an accumulated cost raster

Use the **Cost Distance** tool to calculate the accumulated cost raster. Provide the slope raster as the cost surface and specify the starting point (e.g., one of the cabins). This step computes the cumulative cost of traveling across the terrain.

## Joining rasters through basic arithmetic

Combine multiple rasters (e.g., slope, barriers) using the **Raster Calculator** tool. For instance, add or multiply rasters to create a composite cost surface that incorporates multiple factors influencing travel difficulty.

## Finding the least cost path between two points

Use the **Cost Path** tool to determine the least-cost path between two points. Provide the accumulated cost raster and the destination point as inputs. The output is a polyline representing the optimal path based on the cost surface.


# Results (/Analysis)

# Discussion (/Analysis)

# Conclusion/Summary

# References
